In [4]:
!pip install -q transformers
!pip install -q datasets torch
!pip install -q torch

In [7]:
import torch
from datasets import load_dataset
from transformers import AlbertForSequenceClassification
from transformers import AlbertTokenizer
from transformers import TrainingArguments
from transformers import Trainer
from sklearn.metrics import accuracy_score
from transformers.utils import logging

In [9]:
# use colab T4 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load GLUE QNLI dataset and ALBERT tokenizer
dataset = load_dataset("glue", "qnli")

tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

# tokenize dataset
def preprocess_function(examples):
    return tokenizer(examples["question"], examples["sentence"],
                     padding="max_length", truncation=True, max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

# label mapping - 0 = "not entailment", 1 = "entailment"
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

model = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2).to(device)

# training arguments
training_args = TrainingArguments(
    output_dir="./albert_qnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=500,
    load_best_model_at_end=True,
    report_to="none"
)

# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

# trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

logging.set_verbosity_error()

trainer.train()

# model accuracy evaluation
eval_results = trainer.evaluate()
print(f"Validation Accuracy: {eval_results['eval_accuracy']:.4f}")

model.save_pretrained("./albert_qnli_finetuned")
tokenizer.save_pretrained("./albert_qnli_finetuned")


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-b9259a69e588>:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.512800,0.483053,0.782537
2,0.428400,0.416606,0.815669
3,0.379500,0.382413,0.832876


Validation Accuracy: 0.8329


('./albert_qnli_finetuned/tokenizer_config.json',
 './albert_qnli_finetuned/special_tokens_map.json',
 './albert_qnli_finetuned/spiece.model',
 './albert_qnli_finetuned/added_tokens.json')